### Polynomial Chaos Expansion example: Exponential function (3 random inputs, scalar output)

Authors: Katiana Kontolati, Dimitrios Loukrezis \
Date: May 7, 2021

In this example, PCE is used to generate a surrogate model for a given set of 3D data.

### Dette & Pepelyshev exponential function

$$ f(x) = 100(\exp{(-2/x_1^{1.75})} + \exp{(-2/x_2^{1.5})} + \exp{(-2/x_3^{1.25})})$$

**Description:**  Dimensions: 3

**Input Domain:**  This function is evaluated on the hypercube $x_i \in [0,1]$ for all i = 1,2,3.

**Reference:**  Dette, H., & Pepelyshev, A. (2010). Generalized Latin hypercube design for computer experiments. Technometrics, 52(4).

Import necessary libraries.

In [1]:
import math
import numpy as np
import matplotlib.pyplot as plt
from UQpy.distributions import Uniform, JointIndependent
from UQpy.surrogates import *

Define the function.

In [2]:
def function(x):
    return 100*(np.exp(-2/(x[:,0]**1.75)) + np.exp(-2/(x[:,1]**1.5)) + np.exp(-2/(x[:,2]**1.25)))

Define the input probability distributions.

In [3]:
# input distributions
dist = Uniform(loc=0, scale=1)   
marg = [dist]*3
joint = JointIndependent(marginals=marg)

Compute reference mean and variance values using Monte Carlo sampling.

In [4]:
# reference moments via Monte Carlo Sampling
n_samples_mc = 1000000
xx = joint.rvs(n_samples_mc)
yy = function(xx)
mean_ref = yy.mean()
var_ref = yy.var()

Create validation data sets, to be used later to estimate the accuracy of the PCE.

In [5]:
# validation data sets
n_samples_val = 1000
xx_val = joint.rvs(n_samples_val)
yy_val = function(xx_val)

Assess the PCE in terms of approximation and moment estimation accuracy, for increasing maximum polynomial degree.

In [6]:
# construct PCE surrogate models
l2_err = []
mean_err = []
var_err = []
for max_degree in range(1,10):
    print(' ')
    
    # PCE basis
    print('Total degree: ', max_degree)
    polynomial_basis = PolynomialBasis.create_total_degree_basis(joint, max_degree)
    print('Size of basis:', polynomial_basis.polynomials_number)
    
    # generate training data
    sampling_coeff = 5
    print('Sampling coefficient: ', sampling_coeff)
    np.random.seed(42)
    n_samples = math.ceil(sampling_coeff*polynomial_basis.polynomials_number)
    print('Training data: ', n_samples)
    xx_train = joint.rvs(n_samples)
    yy_train = function(xx_train)
    
    # fit model
    least_squares = LeastSquareRegression()
    pce_metamodel = PolynomialChaosExpansion(polynomial_basis=polynomial_basis, regression_method=least_squares)
    pce_metamodel.fit(xx_train, yy_train)
    
    # validation errors
    yy_val_pce = pce_metamodel.predict(xx_val)
    errors = np.abs(yy_val - yy_val_pce.flatten())
    error_l2 = np.linalg.norm(errors)
    l2_err.append(error_l2)
    print('Validation error in the l2 norm:', error_l2)
    
    # moment errors
    pce_moments = pce_metamodel.get_moments()
    mean_pce=pce_moments[0]
    var_pce = pce_moments[1]
    mean_err.append(np.abs((mean_pce - mean_ref)/mean_ref))
    var_err.append(np.abs((var_pce - var_ref)/var_ref))
    print('Relative error in the mean:', mean_err[-1])
    print('Relative error in the variance:', var_err[-1])

 
Total degree:  1
Size of basis: 4
Sampling coefficient:  5
Training data:  20
Validation error in the l2 norm: 128.2510771998685
Relative error in the mean: 0.06891993802615205
Relative error in the variance: 0.1917859074532558
 
Total degree:  2
Size of basis: 10
Sampling coefficient:  5
Training data:  50
Validation error in the l2 norm: 33.365441908631695
Relative error in the mean: 0.007290853863680285
Relative error in the variance: 0.025065723446381753
 
Total degree:  3
Size of basis: 20
Sampling coefficient:  5
Training data:  100
Validation error in the l2 norm: 11.061272098645427
Relative error in the mean: 0.005328221342681636
Relative error in the variance: 0.0320018028415459
 
Total degree:  4
Size of basis: 35
Sampling coefficient:  5
Training data:  175
Validation error in the l2 norm: 6.501964542859614
Relative error in the mean: 0.0002762598534443847
Relative error in the variance: 0.006443187535350117
 
Total degree:  5
Size of basis: 56
Sampling coefficient:  5
Tra

/Users/dimitris/PycharmProjects/dimtsap/UQpy/src/UQpy/surrogates/polynomial_chaos/regressions/LeastSquareRegression.py:18: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  c_, res, rank, sing = np.linalg.lstsq(design_matrix, y)


Validation error in the l2 norm: 0.1536337143969933
Relative error in the mean: 0.00043375505574659795
Relative error in the variance: 0.0012499413045997745
